In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize, Resize, RandomRotation
import numpy as np
from torch.utils.data import DataLoader
from modules.deep_pixel_wise.Dataset_v2 import PixWiseDataset
from modules.deep_pixel_wise.Model import DeePixBiS
from modules.deep_pixel_wise.Loss import PixWiseBCELoss
from modules.deep_pixel_wise.Metrics import predict, test_accuracy, test_loss
from modules.deep_pixel_wise.Trainer import Trainer

In [3]:
train_data = pd.read_csv('.\data\\path_files\\train_data_celeb_nuaa.csv').reset_index(drop=True)
test_data = pd.read_csv('.\data\\path_files\\test_data_celeb_nuaa.csv').reset_index(drop=True)
val_data = pd.read_csv('.\data\\path_files\\val_data_celeb_nuaa.csv').reset_index(drop=True)

In [4]:
train_data_shuffled = train_data.sample(frac=1).reset_index(drop=True)
train_data_shuffled.head()

,name,label
0,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,1
1,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0
2,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0
3,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0
4,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0


In [5]:
train_data_shuffled.shape

(7022, 2)

In [6]:
val_data.shape

(878, 2)

In [7]:
test_data.shape

(878, 2)

In [9]:
int(len(train_data_shuffled)*0.5)

3511

In [8]:
sub_train_data = train_data_shuffled[:int(len(train_data_shuffled)*0.5)].reset_index(drop=True)
sub_train_data.head()

,name,label
0,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0
1,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0
2,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,1
3,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0
4,C:\KhoiNXM\Workspace\Learning\Master Thesis\De...,0


In [15]:
# Setting device on GPU if available, else CPU
import torch

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated: ', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached: ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    print('Max memory reserved: ', round(torch.cuda.max_memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA T1200 Laptop GPU
Memory Usage:
Allocated:  0.0 GB
Cached:  0.0 GB
Max memory reserved:  0.0 GB


In [ ]:
model = DeePixBiS()
# model.load_state_dict(torch.load('/content/drive/MyDrive/FPT MSE/Capstone Project/Anti_Spoof_DPW/DeePixBiS.pth'))
loss_fn = PixWiseBCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.0001)
train_tfms = Compose([Resize([224, 224]),
                      RandomHorizontalFlip(),
                      RandomRotation(10),
                      ToTensor(),
                    #   Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                      Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_tfms = Compose([Resize([224, 224]),
                     ToTensor(),
                    #  Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                     Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = PixWiseDataset(train_data_shuffled, transform=train_tfms)
train_ds = train_dataset.dataset()

val_dataset = PixWiseDataset(val_data, transform=test_tfms)
val_ds = val_dataset.dataset()

batch_size = 16
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)

trainer = Trainer(train_dl, val_dl, model, 5, opt, loss_fn, device)

print('Training Beginning\n')
trainer.fit()

print('\nTraining Complete')
torch.save(model.state_dict(), './models/DeePixBiS/DeePixBiS_celeb_nuaa_130223.pth')

In [ ]:
del model, train_dataset, train_ds, train_dl, val_dataset, val_ds, val_dl, trainer
sub_train_data_part2 = train_data_shuffled[int(len(train_data_shuffled)*0.5:)].reset_index(drop=True)

In [ ]:
model = DeePixBiS()
model.load_state_dict(torch.load('/content/drive/MyDrive/FPT MSE/Capstone Project/Anti_Spoof_DPW/DeePixBiS.pth'))
loss_fn = PixWiseBCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.0001)
train_tfms = Compose([Resize([224, 224]),
                      RandomHorizontalFlip(),
                      RandomRotation(10),
                      ToTensor(),
                    #   Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                      Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

test_tfms = Compose([Resize([224, 224]),
                     ToTensor(),
                    #  Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
                     Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = PixWiseDataset(train_data_shuffled, transform=train_tfms)
train_ds = train_dataset.dataset()

val_dataset = PixWiseDataset(val_data, transform=test_tfms)
val_ds = val_dataset.dataset()

batch_size = 16
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)

trainer = Trainer(train_dl, val_dl, model, 5, opt, loss_fn, device)

print('Training Beginning\n')
trainer.fit()

print('\nTraining Complete')
torch.save(model.state_dict(), './models/DeePixBiS/DeePixBiS_celeb_nuaa_130223.pth')